In [ ]:
import pymongo
from prettytable import PrettyTable
from geopy.distance import geodesic
from datetime import datetime

# Connessione a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["concertsDB"]
concerts_collection = db["concerts"]
purchases_collection = db["purchases"]

def add_concert():
    name = input("Inserisci il nome del concerto: ")
    artist = input("Inserisci il nome dell'artista: ")
    date = input("Inserisci la data del concerto (YYYY-MM-DD): ")
    price = float(input("Inserisci il prezzo del biglietto: "))
    location = input("Inserisci la posizione del concerto (latitudine, longitudine): ").split(',')
    latitude = float(location[0])
    longitude = float(location[1])
    concert = {
        "name": name,
        "artist": artist,
        "date": date,
        "price": price,
        "available_tickets": 100,  # Numero di biglietti disponibili
        "location": {"latitude": latitude, "longitude": longitude}
    }
    concerts_collection.insert_one(concert)
    print("Concerto aggiunto con successo!")

def view_concerts():
    concerts = concerts_collection.find()
    table = PrettyTable(["ID", "Nome", "Artista", "Data", "Prezzo", "Biglietti Disponibili", "Posizione"])
    for concert in concerts:
        location = f"{concert['location']['latitude']}, {concert['location']['longitude']}"
        table.add_row([str(concert["_id"]), concert["name"], concert["artist"], concert["date"], concert["price"], concert["available_tickets"], location])
    print(table)

def search_concert():
    print("Cerca un concerto per:")
    print("1. Artista")
    print("2. Nome del concerto")
    print("3. Intervallo di date (opzionale)")
    print("4. Vicinanza (opzionale)")
    choice = input("Scegli un'opzione: ")

    query = {}
    if choice == "1":
        artist = input("Inserisci il nome dell'artista: ")
        query["artist"] = {"$regex": artist, "$options": "i"}
    elif choice == "2":
        name = input("Inserisci il nome del concerto: ")
        query["name"] = {"$regex": name, "$options": "i"}
    elif choice == "3":
        start_date = input("Inserisci la data di inizio (YYYY-MM-DD): ")
        end_date = input("Inserisci la data di fine (YYYY-MM-DD): ")
        query["date"] = {"$gte": start_date, "$lte": end_date}
    elif choice == "4":
        location = input("Inserisci la tua posizione attuale (latitudine, longitudine): ").split(',')
        latitude = float(location[0])
        longitude = float(location[1])
        radius_km = 7

        concerts = concerts_collection.find()
        table = PrettyTable(["ID", "Nome", "Artista", "Data", "Prezzo", "Biglietti Disponibili", "Distanza (km)"])
        for concert in concerts:
            concert_location = (concert["location"]["latitude"], concert["location"]["longitude"])
            user_location = (latitude, longitude)
            distance = geodesic(concert_location, user_location).km
            if distance <= radius_km:
                table.add_row([str(concert["_id"]), concert["name"], concert["artist"], concert["date"], concert["price"], concert["available_tickets"], round(distance, 2)])
        print(table)
        return

    concerts = concerts_collection.find(query)
    table = PrettyTable(["ID", "Nome", "Artista", "Data", "Prezzo", "Biglietti Disponibili"])
    for concert in concerts:
        table.add_row([str(concert["_id"]), concert["name"], concert["artist"], concert["date"], concert["price"], concert["available_tickets"]])
    print(table)

def purchase_ticket():
    concert_id = input("Inserisci l'ID del concerto che vuoi acquistare: ")
    concert = concerts_collection.find_one({"_id": pymongo.ObjectId(concert_id)})
    if concert:
        if concert["available_tickets"] > 0:
            name = input("Inserisci il tuo nome: ")
            purchase = {
                "concert_id": concert["_id"],
                "concert_name": concert["name"],
                "buyer_name": name
            }
            purchases_collection.insert_one(purchase)
            concerts_collection.update_one({"_id": concert["_id"]}, {"$inc": {"available_tickets": -1}})
            print("Biglietto acquistato con successo!")
        else:
            print("Mi dispiace, non ci sono più biglietti disponibili per questo concerto.")
    else:
        print("Concerto non trovato.")

def main():
    while True:
        print("\nGestione Concerti CLI")
        print("1. Aggiungi Concerto")
        print("2. Visualizza Concerti")
        print("3. Ricerca Concerto")
        print("4. Acquista Biglietto")
        print("5. Esci")
        choice = input("Scegli un'opzione: ")
        if choice == "1":
            add_concert()
        elif choice == "2":
            view_concerts()
        elif choice == "3":
            search_concert()
        elif choice == "4":
            purchase_ticket()
        elif choice == "5":
            break
        else:
            print("Opzione non valida. Riprova.")

if __name__ == "__main__":
    main()
